In [2]:
import pyaudio
import os
import numpy as np
import time

# constants
CHUNK = 1024             # samples per frame
FORMAT = pyaudio.paInt16    # audio format (bytes per sample?)
CHANNELS = 1                 # single channel for microphone
RATE = 44100                 # samples per second

In [3]:
import sys
import serial

serialPort = '/dev/cu.usbmodem14301'
baudRate = 115200
usbConnection = serial.Serial(serialPort, baudRate)

def commandArduino(string):
    outputStr = string + '\n'
    outputStr = bytes(outputStr, 'utf-8')
    usbConnection.write(outputStr)
    usbConnection.reset_input_buffer()

# Figure out how to auto-scale sensitivity
    - Use a running average
    - Take into account how much is being clipped at the upper range and feed it back into the delta modifier
    
## Or maybe leave sensitivity constant with manual adjustment and allow the user to naturally ignore or pay attention to vibration
## But should avoid clipping upper range, so that loudest sounds always stand out

In [55]:
class runningAverage:
    def __init__(self, limit):
        self.arr = np.array([] * limit)
        self.limit = limit
    def append(self, value):
        if len(self.arr) >= self.limit:
            self.arr = self.arr[1:]
        self.arr = np.append(self.arr, value)
    def average(self):
        return np.average(self.arr)

In [59]:
# pyaudio class instance
p = pyaudio.PyAudio()

# stream object to get data from microphone
stream = p.open(
    format=FORMAT,
    channels=CHANNELS,
    rate=RATE,
    input=True,
    output=True,
    frames_per_buffer=CHUNK
)

# for measuring frame rate
frame_count = 0
frame_limit = 100
start_time = time.time()

mod_value = 0.7
mod_average = runningAverage(100)

while True:
    data = stream.read(CHUNK, exception_on_overflow = False)
    data_np = np.frombuffer(data, dtype=np.int16)
    
    # volume
    delta = np.max(data_np) - np.min(data_np)
    delta_mod = int(np.abs(delta) ** mod_value)
    delta_bounded = min(max(delta_mod, 0), 128)
    
    mod_bounded_diff = delta_mod - delta_bounded
    mod_average.append(mod_bounded_diff ** 0.25)
    mod_value = (1 / (mod_average.average() + 4)) + 0.5

    commandArduino(f'vibrate,{delta_bounded}')
    print(delta, delta_bounded, mod_value)
    
    frame_count += 1

end_time = time.time()
print(frame_limit / (end_time - start_time))

0 0 0.75
0 0 0.75
0 0 0.75
0 0 0.75
0 0 0.75
0 0 0.75
0 0 0.75
0 0 0.75
0 0 0.75
0 0 0.75
0 0 0.75
34 14 0.75
26 11 0.75
11 6 0.75
9 5 0.75
8 4 0.75
924 128 0.7411381742268414
147 40 0.7416139833434521
64 21 0.74204130174092
23 10 0.7424271827003021
834 128 0.7370556415200096
223 53 0.7376148728097963
45 16 0.7381277846184957
20 9 0.7385999028579517
10 5 0.7390359080695406
13 6 0.7394397911282699
61 20 0.7398149757556103
10 5 0.740164415812423
78 25 0.7404906732279901
53 18 0.740795980917047
14 7 0.7410822939537066
10 5 0.7413513314822116
11 5 0.7416046112634463
9 5 0.7418434783239086
9 5 0.742069128849231
11 5 0.7422826302183789
9 5 0.742484937886719
8 4 0.7426769096814259
10 5 0.7428593179604137
9 5 0.7430328599982556
8 4 0.7431981668935643
8 4 0.7433558112377092
9 5 0.7435063137412725
10 5 0.7436501489798514
9 5 0.7437877503927875
7 4 0.7439195146457483
8 4 0.7440458054496465
9 5 0.7441669569133323
9 5 0.7442832764951404
9 5 0.744395047608191
8 4 0.744502531925926
9 5 0.744605971427